In [ ]:
import matplotlib.animation as animation
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
from ipywidgets import interact
from scipy.optimize import curve_fit

# peak1 = (1380, 1410)
# peak2 = (730, 770)
# peak3 = (1300, 1380)

# map_a.draw_curve(lorentz, (1392, 2, -1.45 * 10**6, -7000), 1330, 1450, base_vector = 0)

In [ ]:
def lorentz(X, mean, sigma, scale, offset):
    return scale/(np.pi * sigma * (1 + ((X - mean)/sigma)**2)) + offset

def lorentz_shift(X, mean, sigma, scale, offset):
    return -(2 * sigma * (-mean + X)) / (np.pi * (mean**2 + sigma**2 - 2*mean*X + np.power(X, 2))**2)*scale + offset

def both_lorentz(X, mean1, sigma1, scale1, offset1, mean2, sigma2, scale2, offset2):
    return lorentz(X, mean1, sigma1, scale1, offset1) + lorentz_shift(X, mean2, sigma2, scale2, offset2)

class Map:
    def __init__(self, src, x, y):
        raw_data = np.loadtxt(src, delimiter='\t')
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(raw_data[2:], columns = ['RamanShift'] + list(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T

        self.cov_matrix = np.cov(self.data_matrix)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        
        self.base_vectors = np.dot(self.vectors, self.data_matrix)
    
    def reconstruct(self, count):
        return np.dot(self.vectors[:count].T, self.base_vectors[:count])
    
    def plotComponent(self, num):
        fig = px.imshow(np.reshape(self.vectors[num], self.shape), title ='Komponent ' + str(num))
        return fig
    
    def plotPhysicalComponent(self, v):
        img = np.dot(np.array(v), self.vectors[:len(v)])
        # img = sum([v[i] * self.vectors[i] for i in range(len(v))])
        fig = px.imshow(np.reshape(img, self.shape), title ='Peak')
        return fig
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self, cum):
        if cum:
            fig = px.bar(np.cumsum(self.weights), title = 'Wagi')
        else:
            fig = px.bar(self.weights, title = 'Wagi')
        return fig
    
    def plotSpectrums(self, ids, **kwargs):
        fig = go.Figure()
        if 'base' not in kwargs:
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = self.data_matrix[num], mode='lines', name = 'ID: ' + str(num))
        else:
            r = self.reconstruct(kwargs['base'])
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = r[num], mode='lines', name = 'ID: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
    
    def plotSpectrum(self, num, **kwargs):
        return self.plotSpectrums([num], **kwargs)
    
    def plotBaseVectors(self, count):
        fig = go.Figure()
        for num in range(count):
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[num], mode='lines', name = 'Num: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
            
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: np.linalg.norm([x, y] - e), self.map)))
    
    def posToId(self, x, y):
        return self.shape[0] * y + x
    
    def draw_curve(self, curve, params, a, b, **draw):
        fig = go.Figure()
        if 'spectrum' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.data_matrix[draw['spectrum']], mode='lines')
        if 'base_vector' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[draw['base_vector']], mode='lines')
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        fig.add_scatter(mode='lines', x = data_slice, y = curve(data_slice, *params))
        fig.update_layout(hovermode="x")
        return fig
        
    
map_a = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt', 34, 34)

In [ ]:
# map_a.plotCovMatrix().show()
map_a.plotWeights(True).show()

In [ ]:
map_a.plotSpectrums([0, 10, 78, 150, 450, 765]).show()

In [ ]:
map_a.plotBaseVectors(5)

In [ ]:
r = map_a.reconstruct(5)
d = map_a.data_matrix - r
i = 78
f = map_a.plotSpectrum(i)
f.add_scatter(x = map_a.raman_shift, y = r[i], mode='lines', name = 'Rekonstrukcja')
f.add_scatter(x = map_a.raman_shift, y = d[i], mode='lines', name = 'Roznica')
f.update_layout(hovermode="x")
f.show()

In [ ]:
i = 1
px.line(x = map_a.raman_shift, y = map_a.base_vectors[i]).show()
map_a.plotComponent(i).show()

In [ ]:
d = map_a.data_matrix - map_a.reconstruct(5)
img = np.reshape(np.sum(np.abs(d), axis = 1), (34, 34))
px.imshow(img)

In [ ]:
map_a.plotSpectrum(map_a.posToId(6, 15))

In [ ]:
map_a.draw_curve(lorentz, (750, 5, -1.3 * 10**5, -5000), 730, 770, base_vector = 0)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, 1.3 * 10**4, 1800), 730, 770, base_vector = 1)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, -2.2 * 10**4, -600), 730, 770, base_vector = 2)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, -9.5 * 10**4, -4000), 730, 770, base_vector = 3)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, 2.5 * 10**4, -2300), 730, 770, base_vector = 4)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, -1.3 * 10**4, -1100), 730, 770, base_vector = 5)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, 1 * 10**4, 900), 730, 770, base_vector = 6)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, -5.5 * 10**4, 100), 730, 770, base_vector = 7)

In [ ]:
map_a.draw_curve(lorentz, (750, 5.5, 8 * 10**3, 2500), 730, 770, base_vector = 8)


In [ ]:
peak1 = [-1.3 * 10**5, 1.3 * 10**4, -2.2 * 10**4, -9.5 * 10**4, 2.5 * 10**4, -1.3 * 10**4, 1 * 10**4, -5.5 * 10**4, 8 * 10**3]
map_a.plotPhysicalComponent(peak1).show()


In [ ]:
map_a.plotSpectrums([map_a.posToId(6 ,  15),
                     map_a.posToId(6 ,  11),
                     map_a.posToId(23, 15)
                     ]).show()

In [ ]:
px.line(x = map_a.raman_shift, y = np.mean(map_a.data_matrix, axis = 0))

In [ ]:
map_a.draw_curve(lorentz, (1345, 17, -7.5 * 10**5, -7000), 1320, 1370, base_vector = 0)


In [ ]:
map_a.draw_curve(lorentz, (1345, 17, 8 * 10**4, -3100), 1320, 1370, base_vector = 1)


In [363]:
map_a.draw_curve(lorentz, (1345, 17, 10 * 10**4, -1000), 1320, 1370, base_vector = 4)


In [379]:
map_a.draw_curve(lorentz, (1345, 17, 20 * 10**4, -1700), 1320, 1370, base_vector = 5)


In [387]:
map_a.draw_curve(lorentz, (1345, 17, -11 * 10**4, 2100), 1320, 1370, base_vector = 6)


In [388]:
peak2 = [-7.5 * 10**5, 0, 0, 8 * 10**4, 10 * 10**4, 20 * 10**4, -11 * 10**4]
map_a.plotPhysicalComponent(peak2).show()

In [390]:
map_a.plotSpectrums([map_a.posToId(17 ,  13),
                     map_a.posToId(30 ,  3),
                     map_a.posToId(30, 27),
                     map_a.posToId(6, 14)
                     ]).show()

In [401]:
map_a.draw_curve(lorentz, (1392, 2.5, -17 * 10**5, -7000), 1360, 1420, base_vector = 0)


In [407]:
map_a.draw_curve(lorentz, (1392, 2.5, 8 * 10**4, -2800), 1360, 1420, base_vector = 1)

In [411]:
map_a.draw_curve(lorentz, (1392, 2.5, 4 * 10**4, -3330), 1360, 1420, base_vector = 3)

In [415]:
map_a.draw_curve(lorentz, (1392, 2.5, -5 * 10**4, -100), 1360, 1420, base_vector = 4)

In [419]:
map_a.draw_curve(lorentz, (1392, 2.5, -3.7 * 10**4, 0), 1360, 1420, base_vector = 5)

In [ ]:
peak3 = [-17 * 10**5, 8 * 10**4, 0, 4 * 10**4, -5 * 10**4, -3.7 * 10**4]